# **import libraries**

In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# **Download Data and Read a head**

In [6]:
df = pd.read_csv("cases_expanded.csv", encoding="utf-8")
df.head()

,case_id,description,channel,created_at,person_id,duplicate_of
0,3001,انا موقوف في الحائر واطلب افادة عن وضعي القانوني,Website,3/3/2025,P2939,NaN
1,3002,احتجاز دون محاكمة في سجن الحائر وأحتاج متابعة ...,Website,3/9/2025,P1760,3001.0
2,3003,سجن الحائر يحتجزني بلا حكم وأحتاج متابعة محامي,MobileApp,3/14/2025,P1134,3001.0
3,3004,سجن الحائر يحتجزني بلا حكم وأحتاج متابعة محامي,Website,2/21/2025,P1088,3001.0
4,3005,تم احتجازي بسجن الحائر من غير صدور حكم حتى الآن,WalkIn,1/28/2025,P1203,3001.0


In [7]:
df["duplicate_of"].value_counts()


,count
duplicate_of,
3001.0,149
3151.0,149
3301.0,149
3451.0,149
3601.0,149
3751.0,149


# **cleaning data -> Preprocessing**

In [8]:
import re

def clean_text(text):
    # إزالة التشكيل
    text = re.sub(r'[ًٌٍَُِّْـ]', '', text)
    # إزالة أي رموز أو أرقام
    text = re.sub(r'[^ء-ي\s]', '', text)
    # إزالة المسافات الزايدة
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['clean_description'] = df['description'].apply(clean_text)
df[['description', 'clean_description']].head()


,description,clean_description
0,انا موقوف في الحائر واطلب افادة عن وضعي القانوني,انا موقوف في الحائر واطلب افادة عن وضعي القانوني
1,احتجاز دون محاكمة في سجن الحائر وأحتاج متابعة ...,احتجاز دون محاكمة في سجن الحائر وأحتاج متابعة ...
2,سجن الحائر يحتجزني بلا حكم وأحتاج متابعة محامي,سجن الحائر يحتجزني بلا حكم وأحتاج متابعة محامي
3,سجن الحائر يحتجزني بلا حكم وأحتاج متابعة محامي,سجن الحائر يحتجزني بلا حكم وأحتاج متابعة محامي
4,تم احتجازي بسجن الحائر من غير صدور حكم حتى الآن,تم احتجازي بسجن الحائر من غير صدور حكم حتى الآن


# **Vectorization -> conv char to num**( feature extract)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

# تعريف المحول
vectorizer = TfidfVectorizer()

# تحويل النصوص (clean_description) إلى متجهات رقمية
X = vectorizer.fit_transform(df['clean_description'])

print("عدد البلاغات:", X.shape[0])
print("عدد الكلمات (المميزات):", X.shape[1])


عدد البلاغات: 1000
عدد الكلمات (المميزات): 331


# **Cosine Similarity**

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

# نحسب مصفوفة التشابه
similarity_matrix = cosine_similarity(X)

# نعرض جزء منها (أول 5 × 5)
print(similarity_matrix[:5, :5])


[[1.         0.10431278 0.06039017 0.06039017 0.05428427]
 [0.10431278 1.         0.40177451 0.40177451 0.05587986]
 [0.06039017 0.40177451 1.         1.         0.14227429]
 [0.06039017 0.40177451 1.         1.         0.14227429]
 [0.05428427 0.05587986 0.14227429 0.14227429 1.        ]]


# **Mark duplicates using threshold**

In [11]:
import numpy as np

threshold = 0.80

is_dup = np.zeros(len(df), dtype=bool)
best_match_case_id = np.full(len(df), -1, dtype=int)
best_match_score = np.zeros(len(df))

for i in range(len(df)):
    sims = similarity_matrix[i].copy()
    sims[i] = -1  # ignore itself
    j = sims.argmax()  # best match
    if sims[j] >= threshold:
        is_dup[i] = True
        best_match_case_id[i] = int(df.iloc[j]['case_id'])
        best_match_score[i] = sims[j]

df['is_duplicate_pred']   = is_dup
df['best_match_case_id']  = best_match_case_id
df['best_match_score']    = best_match_score

df[['case_id','is_duplicate_pred','best_match_case_id','best_match_score']].head(10)


,case_id,is_duplicate_pred,best_match_case_id,best_match_score
0,3001,True,3007,1.0
1,3002,True,3013,1.0
2,3003,True,3004,1.0
3,3004,True,3003,1.0
4,3005,True,3017,1.0
5,3006,True,3003,1.0
6,3007,True,3001,1.0
7,3008,True,3009,1.0
8,3009,True,3008,1.0
9,3010,True,3001,1.0


# **Find top-k similar cases (by index)**

In [12]:
def top_k_for_index(idx, k=5):
    sims = similarity_matrix[idx].copy()
    sims[idx] = -1
    top_idx = sims.argsort()[::-1][:k]
    out = df.loc[top_idx, ['case_id','description','channel','created_at']].copy()
    out['similarity'] = sims[top_idx]
    return out

top_k_for_index(0, k=5)


,case_id,description,channel,created_at,similarity
88,3089,انا موقوف في الحائر واطلب افادة عن وضعي القانوني,MobileApp,1/7/2025,1.0
57,3058,انا موقوف في الحائر واطلب افادة عن وضعي القانوني,WalkIn,3/16/2025,1.0
62,3063,انا موقوف في الحائر واطلب افادة عن وضعي القانوني,Website,2/17/2025,1.0
64,3065,انا موقوف في الحائر واطلب افادة عن وضعي القانوني,MobileApp,1/4/2025,1.0
110,3111,انا موقوف في الحائر واطلب افادة عن وضعي القانوني,PhoneCall,2/15/2025,1.0


# **Evaluate with Precision@k / Recall@k**

In [17]:
def precision_recall_at_k(sim_mx, df, k=3):
    precs, recs = [], []
    for i in range(len(df)):
        sims = sim_mx[i].copy(); sims[i] = -1
        top_idx = sims.argsort()[::-1][:k]
        suggested_ids = set(df.iloc[top_idx]['case_id'].astype(int))

        this = df.iloc[i]
        base = int(this['duplicate_of']) if str(this['duplicate_of']).strip() else int(this['case_id'])

        gt_ids = set()
        for _, row in df.iterrows():
            row_base = int(row['duplicate_of']) if str(row['duplicate_of']).strip() else int(row['case_id'])
            if row_base == base and int(row['case_id']) != int(this['case_id']):
                gt_ids.add(int(row['case_id']))

        if len(gt_ids) == 0:
            recs.append(1.0);  continue

        tp = len(suggested_ids & gt_ids)
        precs.append(tp/len(suggested_ids) if suggested_ids else 0.0)
        recs.append(tp/len(gt_ids) if gt_ids else 0.0)

    return (np.mean(precs) if precs else 0.0,
            np.mean(recs)  if recs  else 0.0)


# **Show top 10 predicted duplicates with their best matches**

In [19]:
duplicates = df[df['is_duplicate_pred'] == True].head(10)

for idx, row in duplicates.iterrows():
    print("🔹 Case:", row['case_id'])
    print("Text :", row['description'])
    print("Predicted duplicate of:", row['best_match_case_id'])
    print("Similarity score:", round(row['best_match_score'], 2))
    print("Matching text:", df.loc[df['case_id'] == row['best_match_case_id'], 'description'].values[0])
    print("-" * 80)


🔹 Case: 3001
Text : انا موقوف في الحائر واطلب افادة عن وضعي القانوني
Predicted duplicate of: 3007
Similarity score: 1.0
Matching text: انا موقوف في الحائر واطلب افادة عن وضعي القانوني
--------------------------------------------------------------------------------
🔹 Case: 3002
Text : احتجاز دون محاكمة في سجن الحائر وأحتاج متابعة عاجلة
Predicted duplicate of: 3013
Similarity score: 1.0
Matching text: احتجاز دون محاكمة في سجن الحائر وأحتاج متابعة عاجلة
--------------------------------------------------------------------------------
🔹 Case: 3003
Text : سجن الحائر يحتجزني بلا حكم وأحتاج متابعة محامي
Predicted duplicate of: 3004
Similarity score: 1.0
Matching text: سجن الحائر يحتجزني بلا حكم وأحتاج متابعة محامي
--------------------------------------------------------------------------------
🔹 Case: 3004
Text : سجن الحائر يحتجزني بلا حكم وأحتاج متابعة محامي
Predicted duplicate of: 3003
Similarity score: 1.0
Matching text: سجن الحائر يحتجزني بلا حكم وأحتاج متابعة محامي
-----------------------

# **save**

In [15]:
df.to_csv("cases_with_predictions.csv", index=False, encoding="utf-8-sig")